In [2]:
import pandas as pd
import gzip
from bs4 import BeautifulSoup
import re
import contractions
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

## READ DATA

In [3]:
# Specify the file path
file_path = 'C:/Users/AAYUSH/python classes/amazon sentiment analysis/amazon_reviews_us_Office_Products_v1_00.tsv.gz'

# Read the compressed TSV file into a DataFrame with 'utf-8' encoding and skip bad lines
with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    df = pd.read_csv(file, sep='\t', error_bad_lines=False)

C:\Users\AAYUSH\AppData\Local\Temp\ipykernel_22512\257221721.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file, sep='\t', error_bad_lines=False)
b'Skipping line 20773: expected 15 fields, saw 22\nSkipping line 39834: expected 15 fields, saw 22\nSkipping line 52957: expected 15 fields, saw 22\nSkipping line 54540: expected 15 fields, saw 22\n'
b'Skipping line 80276: expected 15 fields, saw 22\nSkipping line 96168: expected 15 fields, saw 22\nSkipping line 96866: expected 15 fields, saw 22\nSkipping line 98175: expected 15 fields, saw 22\nSkipping line 112539: expected 15 fields, saw 22\nSkipping line 119377: expected 15 fields, saw 22\nSkipping line 120065: expected 15 fields, saw 22\nSkipping line 124703: expected 15 fields, saw 22\n'
b'Skipping line 134024: expected 15 fields, saw 22\nSkipping line 153938: expected 15 fields, saw 22\nSkipping line 156225: expected 15 

In [4]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31


## Keep Reviews and Ratings

In [5]:
dt=df[['review_body','star_rating']]

In [6]:
dt.head()

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4


In [7]:
dt

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4
...,...,...
2640249,I can't live anymore whithout my Palm III. But...,4
2640250,Although the Palm Pilot is thin and compact it...,4
2640251,This book had a lot of great content without b...,4
2640252,I am teaching a course in Excel and am using t...,5


In [8]:
dt.dropna()

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4
...,...,...
2640249,I can't live anymore whithout my Palm III. But...,4
2640250,Although the Palm Pilot is thin and compact it...,4
2640251,This book had a lot of great content without b...,4
2640252,I am teaching a course in Excel and am using t...,5


 ## We form two classes and select 50000 reviews randomly from each class.


In [9]:
#creating a class column
dt['class'] = dt['star_rating'].map({1: 1, 2: 1, 3: 1, 4: 2, 5: 2})

# Set the number of reviews to select for each rating class
num_reviews_per_class = 50000

# Create a balanced dataset
balanced_dt = dt.groupby('class').sample(n=num_reviews_per_class, random_state=42)

# Shuffle the balanced dataset to mix the rating classes
balanced_dt = balanced_dt.sample(frac=1, random_state=42).reset_index(drop=True)

C:\Users\AAYUSH\AppData\Local\Temp\ipykernel_22512\966436848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['class'] = dt['star_rating'].map({1: 1, 2: 1, 3: 1, 4: 2, 5: 2})


In [10]:
balanced_dt

,review_body,star_rating,class
0,"No frills, basic paper shredder. I shredded ar...",5,2.0
1,I think the G2 is a great pen so I bought this...,5.0,2.0
2,Cannot fill this pencil with a full sized piec...,3,1.0
3,Perfect for what I need! I use it for sorting ...,5,2.0
4,This purchase was something of an impulsive sp...,5,2.0
...,...,...,...
99995,"I have owned an HP printer since 1996, going t...",1,1.0
99996,What a Great pen. Writes smooth and ink is dar...,5,2.0
99997,I really like this keyboard for use with my me...,4.0,2.0
99998,"First off, I have to say I love the size of th...",3,1.0


In [11]:
# Convert the 'review_body' column to strings
balanced_dt['review_body'] = balanced_dt['review_body'].astype(str)

# Calculate the average length of reviews before cleaning
avg_length_before_datacleaning = balanced_dt['review_body'].apply(len).mean()
print(f"Average length of reviews before cleaning: {avg_length_before_datacleaning:.2f} characters")

Average length of reviews before cleaning: 325.75 characters


# Data Cleaning


In [12]:
def preprocess_text(text):
    # Step 0: Convert text to lowercase
    text = text.lower()

    # Step 1: Clean HTML
    def clean_html(html):
        soup = BeautifulSoup(html, "html.parser")
        for data in soup(['style', 'script', 'code', 'a']):
            data.decompose()
        return ' '.join(soup.stripped_strings)
    
    text = clean_html(text)

    # Step 2: Remove punctuation using regular expressions
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

    # Step 3: Remove numbers
    text = re.sub(r'\d+', '', text)

    # Step 4: Remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    # Step 5: Apply contractions
    text = contractions.fix(text)

    return text

In [13]:
# Apply the preprocessing function to the 'review_body' column
balanced_dt['review_body'] = balanced_dt['review_body'].apply(preprocess_text)

# Calculate average length of reviews after cleaning
avg_length_after_cleaning = balanced_dt['review_body'].apply(len).mean()

# Print the results
print(f"{avg_length_before_datacleaning:.2f},{avg_length_after_cleaning:.2f} ")

c:\Users\AAYUSH\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


325.75,309.29 


# Pre-processing

## remove the stop words 

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AAYUSH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AAYUSH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
print(f"Avg length of values before data preprocessing:{avg_length_after_cleaning:.2f} ")

Avg length of values before data preprocessing:309.29 


In [16]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = word_tokenize(text)  # Tokenize the text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [17]:
balanced_dt['review_body'] = balanced_dt['review_body'].apply(remove_stopwords)

## lemmatization

In [18]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [22]:
def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)  # Tokenize the text
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_tokens)

In [23]:
balanced_dt['review_body'] = balanced_dt['review_body'].apply(lemmatize_text)

In [24]:
avg_length_after_preprocessing = balanced_dt['review_body'].apply(len).mean()

In [25]:
print(f"{avg_length_after_cleaning:.2f},{avg_length_after_preprocessing:.2f} ")

309.29,191.23 


# TF-IDF and BoW Feature Extraction

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [27]:
features=balanced_dt['review_body']
labels=balanced_dt['class']

In [28]:
x_train,x_test,y_train,y_test=train_test_split(features,labels,test_size=0.2)

In [32]:
#vectorizer using count vector
vectorizer_01 = CountVectorizer()
features_bow_train = vectorizer_01.fit_transform(x_train)
features_bow_test = vectorizer_01.transform(x_test)

#vectorizer using Tfidf vectorizer
vectorizer_02 = TfidfVectorizer()
features_tfidf_train = vectorizer_01.fit_transform(x_train)
features_tfidf_test = vectorizer_01.transform(x_test)

# 5 Perceptron Using Both Features

In [33]:
# Initialize Perceptron models
perceptron_bow = Perceptron()
perceptron_tfidf = Perceptron()

# Train the Perceptron models
perceptron_bow.fit(features_bow_train, y_train)
perceptron_tfidf.fit(features_tfidf_train, y_train)

# Make predictions on the test data
y_pred_bow = perceptron_bow.predict(features_bow_test)
y_pred_tfidf = perceptron_tfidf.predict(features_tfidf_test)

# Calculate Precision, Recall, and F1-score for BoW and TF-IDF models
precision_bow = precision_score(y_test, y_pred_bow)
recall_bow = recall_score(y_test, y_pred_bow)
f1_bow = f1_score(y_test, y_pred_bow)

precision_tfidf = precision_score(y_test, y_pred_tfidf)
recall_tfidf = recall_score(y_test, y_pred_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

# Print the results in the Jupyter Notebook
print(f"BoW Precision: {precision_bow:.2f}, Recall: {recall_bow:.2f}, F1-score: {f1_bow:.2f}")
print(f"TF-IDF Precision: {precision_tfidf:.2f}, Recall: {recall_tfidf:.2f}, F1-score: {f1_tfidf:.2f}")

BoW Precision: 0.77, Recall: 0.83, F1-score: 0.80
TF-IDF Precision: 0.77, Recall: 0.83, F1-score: 0.80


# SVM

In [34]:
# Initialize Perceptron models
svm_bow = LinearSVC()
svm_tfidf = LinearSVC()

# Train the Perceptron models
svm_bow.fit(features_bow_train, y_train)
svm_tfidf.fit(features_tfidf_train, y_train)

# Make predictions on the test data
y_pred_bow = svm_bow.predict(features_bow_test)
y_pred_tfidf = svm_tfidf.predict(features_tfidf_test)

# Calculate Precision, Recall, and F1-score for BoW and TF-IDF models
precision_bow = precision_score(y_test, y_pred_bow)
recall_bow = recall_score(y_test, y_pred_bow)
f1_bow = f1_score(y_test, y_pred_bow)

precision_tfidf = precision_score(y_test, y_pred_tfidf)
recall_tfidf = recall_score(y_test, y_pred_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

# Print the results in the Jupyter Notebook
print(f"BoW Precision: {precision_bow:.2f}, Recall: {recall_bow:.2f}, F1-score: {f1_bow:.2f}")
print(f"TF-IDF Precision: {precision_tfidf:.2f}, Recall: {recall_tfidf:.2f}, F1-score: {f1_tfidf:.2f}")

c:\Users\AAYUSH\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


BoW Precision: 0.84, Recall: 0.81, F1-score: 0.82
TF-IDF Precision: 0.84, Recall: 0.81, F1-score: 0.82


c:\Users\AAYUSH\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# LOGISTIC REGRESSION

In [35]:
# Initialize and train Logistic Regression models
logistic_regression_bow = LogisticRegression()
logistic_regression_bow.fit(features_bow_train, y_train)

logistic_regression_tfidf = LogisticRegression()
logistic_regression_tfidf.fit(features_tfidf_train, y_train)

# Make predictions
y_pred_bow = logistic_regression_bow.predict(features_bow_test)
y_pred_tfidf = logistic_regression_tfidf.predict(features_tfidf_test)

# Classification report for BoW model
print("BoW Classification Report:")
print(classification_report(y_test, y_pred_bow))

# Classification report for TF-IDF model
print("\nTF-IDF Classification Report:")
print(classification_report(y_test, y_pred_tfidf))

c:\Users\AAYUSH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


BoW Classification Report:
              precision    recall  f1-score   support

         1.0       0.85      0.83      0.84     10002
         2.0       0.83      0.86      0.84      9998

    accuracy                           0.84     20000
   macro avg       0.84      0.84      0.84     20000
weighted avg       0.84      0.84      0.84     20000


TF-IDF Classification Report:
              precision    recall  f1-score   support

         1.0       0.85      0.83      0.84     10002
         2.0       0.83      0.86      0.84      9998

    accuracy                           0.84     20000
   macro avg       0.84      0.84      0.84     20000
weighted avg       0.84      0.84      0.84     20000



c:\Users\AAYUSH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# NAIVE BAYES

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Initialize and train Naive Bayes classifiers
naive_bayes_bow = MultinomialNB()
naive_bayes_bow.fit(features_bow_train, y_train)

naive_bayes_tfidf = MultinomialNB()
naive_bayes_tfidf.fit(features_tfidf_train, y_train)

# Make predictions
y_pred_bow = naive_bayes_bow.predict(features_bow_test)
y_pred_tfidf = naive_bayes_tfidf.predict(features_tfidf_test)

# Classification report for BoW model
print("BoW Classification Report:")
print(classification_report(y_test, y_pred_bow))

# Classification report for TF-IDF model
print("\nTF-IDF Classification Report:")
print(classification_report(y_test, y_pred_tfidf))

BoW Classification Report:
              precision    recall  f1-score   support

         1.0       0.84      0.76      0.80     10002
         2.0       0.78      0.86      0.82      9998

    accuracy                           0.81     20000
   macro avg       0.81      0.81      0.81     20000
weighted avg       0.81      0.81      0.81     20000


TF-IDF Classification Report:
              precision    recall  f1-score   support

         1.0       0.84      0.76      0.80     10002
         2.0       0.78      0.86      0.82      9998

    accuracy                           0.81     20000
   macro avg       0.81      0.81      0.81     20000
weighted avg       0.81      0.81      0.81     20000



In [37]:
print(f"{avg_length_before_datacleaning:.2f},{avg_length_after_cleaning:.2f} ")
print(f"{avg_length_after_cleaning:.2f},{avg_length_after_preprocessing:.2f} ")

print(f"BoW Precision: {precision_bow:.2f}, Recall: {recall_bow:.2f}, F1-score: {f1_bow:.2f}")
print(f"TF-IDF Precision: {precision_tfidf:.2f}, Recall: {recall_tfidf:.2f}, F1-score: {f1_tfidf:.2f}")

print(f"BoW Precision: {precision_bow:.2f}, Recall: {recall_bow:.2f}, F1-score: {f1_bow:.2f}")
print(f"TF-IDF Precision: {precision_tfidf:.2f}, Recall: {recall_tfidf:.2f}, F1-score: {f1_tfidf:.2f}")

325.75,309.29 
309.29,191.23 
BoW Precision: 0.84, Recall: 0.81, F1-score: 0.82
TF-IDF Precision: 0.84, Recall: 0.81, F1-score: 0.82
BoW Precision: 0.84, Recall: 0.81, F1-score: 0.82
TF-IDF Precision: 0.84, Recall: 0.81, F1-score: 0.82


In [38]:
# Classification report for BoW model
print("BoW Classification Report:")
print(classification_report(y_test, y_pred_bow))

# Classification report for TF-IDF model
print("\nTF-IDF Classification Report:")
print(classification_report(y_test, y_pred_tfidf))

BoW Classification Report:
              precision    recall  f1-score   support

         1.0       0.84      0.76      0.80     10002
         2.0       0.78      0.86      0.82      9998

    accuracy                           0.81     20000
   macro avg       0.81      0.81      0.81     20000
weighted avg       0.81      0.81      0.81     20000


TF-IDF Classification Report:
              precision    recall  f1-score   support

         1.0       0.84      0.76      0.80     10002
         2.0       0.78      0.86      0.82      9998

    accuracy                           0.81     20000
   macro avg       0.81      0.81      0.81     20000
weighted avg       0.81      0.81      0.81     20000



In [39]:
print("BoW Classification Report:")
print(classification_report(y_test, y_pred_bow))
print("\nTF-IDF Classification Report:")
print(classification_report(y_test, y_pred_tfidf))

BoW Classification Report:
              precision    recall  f1-score   support

         1.0       0.84      0.76      0.80     10002
         2.0       0.78      0.86      0.82      9998

    accuracy                           0.81     20000
   macro avg       0.81      0.81      0.81     20000
weighted avg       0.81      0.81      0.81     20000


TF-IDF Classification Report:
              precision    recall  f1-score   support

         1.0       0.84      0.76      0.80     10002
         2.0       0.78      0.86      0.82      9998

    accuracy                           0.81     20000
   macro avg       0.81      0.81      0.81     20000
weighted avg       0.81      0.81      0.81     20000

